In [1]:
import numpy as np

In [2]:
class SVM:
    def __init__(self, iter_max=1000, kernel='linear', p=2):
        self.iter_max = iter_max
        self.kernel = kernel
        
    def Kernel(self, i, j):
        if self.kernel == 'linear':
            return sum(self.train_data[i]*self.train_data[j])
        if self.kernel == 'poly':
            return (sum(self.train_data[i]*self.train_data[j]) + 1)**p
        
    def g(self, i):
        s = 0
        for j in len(self.train_data):
            s += self.alpha[j]*self.train_labels[j]*self.Kernel(i, j)
        return s+self.b
    
    def KKT(self, i):
        v = self.train_labels[i]*self.g(i)
        if self.alpha[i] == 0:
            return v >= 1
        if 0<self.alpha[i]<self.C:
            return v == 1
        if self.alpha[i] == self.C:
            return v <= 1
        
    def clip(self, alpha2_new_unc, E1_index, E2_index):
        if self.train_labels[E1_index] != self.train_labels[E2_index]:
            L = max([0, self.alpha[E2_index]-self.alpha[E1_index]])
            H = min([self.C, self.C+self.alpha[E2_index]-self.alpha[E1_index]])
        else:
            L = max([0,self.alpha[E2_index] + self.alpha[E1_index]-self.C])
            H = min([self.C, self.alpha[E2_index]+self.alpha[E1_index]])
        if alpha2_new_unc>H:
            return H
        elif L<=alpha2_new_unc:
            return alpha2_new_unc
        else:
            return L
        
    def fit(self, train_data, train_labels):
        self.train_data = train_data
        self.train_labels = train_labels
        self.alpha = np.ones(len(train_data))
        self.b = 0.0
        self.C = 1.0
        self.E = [self.g(i) - self.train_labels[i] for i in range(len(self.alpha))]
        
        for it in range(self.iter_max):
            priority_index_list = []
            non_priority_index_list = []
            for a in self.alpha:
                if 0<a<self.C:
                    priority_index_list.append(self.alpha.index(a))
                else:
                    non_priority_index_list.append(self.alpha.index(a))
            index_list = priority_index_list.extend(non_priority_index_list)
            
            for E1_index in index_list:
                if not self.KKT(E1_index):
                    if self.E[i]>=0:
                        E1 = self.E[E1_index]
                        E2 = min(self.E)
                        E2_index = self.E.index(E2)
                    else:
                        E1 = self.E[E1_index]
                        E2 = max(self.E)
                        E2_index = self.E.index(E2)
                    eta = self.Kernel(E1_index, E1_index) + self.Kernel(E2_index, E2_index) - 2*self.Kernel(E1_index, E2_index)
                    alpha2_new_unc  =self.alpha[E1_index] + self.train_labels[E1_index]*(E1-E2)/eta
                    alpha2_new = self.clip(alpha2_new_unc, E1_index, E2_index)
                    alpha1_new = self.alpha[E1_index] + self.train_labels[E1_index]*self.train_labels[E2_index]*(self.alpha[E2_index]-alpha2_new)
                    b1_new = -self.E[E1_index]-self.train_labels[E1_index]*self.Kernel(E1_index,E1_index)*(alpha1_new-self.alpha[E1_index])-self.train_labels[E2_index]*self.Kernel(E2_index,E1_index)*(alpha2_new-self.alpha[E2_index])+self.b
                    b2_new = -self.E[E2_index]-self.train_labels[E1_index]*self.Kernel(E1_index,E2_index)*(alpha1_new-self.alpha[E1_index])-self.train_labels[E2_index]*self.Kernel(E2_index,E2_index)*(alpha2_new-self.alpha[E2_index])+self.b
                    
                    if 0<alpha1_new<self.C:
                        b_new = b1_new
                    elif 0<alpha2_new<self.C:
                        b_new = b2_new
                    else:
                        b_new = (b1_new + b2_new)/2
                        
                    self.alpha[E1_index]=alpha1_new
                    self.alpha[E2_index]=alpha2_new
                    self.b = b_new
                    
                    self.E[E1_index] = self.g(E1_index)-self.train_labels[E1_index]
                    self.E[E2_index] = self.g(E2_index)-self.train_labels[E2_index]
               
    def predict(self, test_x):
        y = 0
        for i in len(self.train_data):
            if self.kernel == 'linear':
                y+=self.alpha[i]*self.train_labels[i]*sum(test_x*self.train_data[i])
            if self.kernel == 'poly':
                y+=self.alpha[i]*self.train_labels[i]*(sum(test_x*self.train_data[i])+1)**p
        y = y + self.b
        if y>0: return 1
        else: return -1

SyntaxError: unexpected EOF while parsing (<ipython-input-2-43b70e975821>, line 27)